In [22]:
from selenium import webdriver
import time
import pandas as pd
from scrapy import Selector
from random import randint
import json



In [23]:
# Set the path to the credentials file
credentials_file_path = "../credentials.json"

# Open the file and load the data into a variable
with open(credentials_file_path, "r") as f:
    credentials = json.load(f)

In [24]:
# Get the username and password from the credentials
username = credentials['username']
password = credentials['password']


In [25]:
# Sets up the driver
driver = webdriver.Chrome()

# Opens the login page
url = 'https://www.mobygames.com/user/login/'
driver.get(url)

In [26]:
# Inputs the username and password to login
driver.find_element('id', 'login').send_keys(username)
driver.find_element('id', 'password').send_keys(password)
driver.find_element('css selector', '#main > form > p > button').click() # I am using the full css selector beacuse the only attribute of the button is 'class' and class is not a valid locator

In [27]:
# Go to first page that we scrape
url ='https://www.mobygames.com/platform/xbox/'
driver.get(url)

In [28]:
# Function to scrape all the data I need for one page
def get_all_games(containers): # Only for this first page, i need to change the primary url to the second page and so on... 
    titles = containers.css('.text-nowrap a::text').getall()
    link = containers.css('.text-nowrap a::attr(href)').getall()

    return {
        'title': titles,
        'link': link,
    }

In [29]:
# Creates an empty dataframe that I can add scraped data to
all_games = []
all_games = pd.DataFrame(all_games)

In [30]:
# Runs the code while the 'Next' button is present, when there is a next button, it will click it and scrape the data on the page

while True:
    try:
        sel = Selector(text=driver.page_source)
        containers = sel.css('tbody tr')
        games_to_add = [get_all_games(containers) for containers in sel.css('tbody tr')]
        games_to_add = pd.DataFrame(games_to_add)
        all_games = pd.concat([all_games, games_to_add], axis=0)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        next_button = driver.find_element('xpath', '//p//a[contains(text(), "Next")]')
        time.sleep(randint(2, 4))
        next_button.click()

    except:
        print("No 'Next' button found. Exiting loop.")
        break

No 'Next' button found. Exiting loop.


In [31]:
# Adds all_games to a csv file
all_games.to_csv("../data/raw data/all_games.csv", index=False)